In [1]:
import scipy.io as scio
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import torch
from torch import nn

In [3]:
raw = scio.loadmat(r'C:\Users\qwyni\OneDrive\RepeatThesis\100x5.mat')
T,D,WL=raw['TS'][0],raw['D'][0],raw['lambda'][0]

In [4]:
T=np.array([i[0] for i in T],dtype=np.float32)
D=np.array([i[0] for i in D],dtype=np.float32)
len(D),len(T)

(1000000, 1000000)

In [ ]:
fig, ax = plt.subplots(1, 5,figsize=(20,6))
for i in range(10):
    ax.flat[i].plot(WL,T[i],label='T')
    ax.flat[i].legend()
    ax.flat[i].set_title('Total')

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
T=torch.tensor(T).to(device)
D=torch.tensor(D).to(device)

In [ ]:
from torch.utils.data import DataLoader,Dataset
class Mydata(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
    def __len__(self):
        return len(self.label)
    def __getitem__(self, item):
        return self.data[item], self.label[item]

idx=len(T)//5
train_dataset = Mydata(D[idx:],T[idx:])
test_dataset = Mydata(D[:idx],T[:idx])
train = DataLoader(train_dataset, batch_size=256, shuffle=True)
test = DataLoader(test_dataset, batch_size=2048, shuffle=False)

In [ ]:
model=nn.Sequential(nn.Linear(10,512),
             nn.Tanhshrink(),
             nn.Linear(512,256),
             nn.Tanhshrink(),
             nn.Linear(256,128),
             nn.Tanhshrink(),
             nn.Linear(128,71))
criterion=nn.MSELoss(reduction='mean')
lr=3e-4
opti=torch.optim.NAdam(model.parameters(),lr=lr)
model=model.to(device)
train_loss,test_loss=[],[]

In [ ]:
eps=5000
for ep in range(1+eps):
    acc_sum,loss_sum,train_tot=0,0,0
    for i,(x_train,y_train) in enumerate(train):
        opti.zero_grad()
        y_pred=model(x_train)
        loss=criterion(y_pred,y_train)
        loss.backward()
        opti.step()
        loss_sum+=loss.item()*len(x_train)
        train_tot+=len(x_train)
    train_loss.append(loss_sum/train_tot)
    with torch.no_grad():
        acc_sum,loss_sum,test_tot=0,0,0
        for x_test,y_test in test:
            y_pred=model(x_test)
#             acc+=torch.sum(y_pred.argmax(axis=1)==y_test).item()
            loss=criterion(y_pred,y_test)
            loss_sum+=loss.item()*len(x_test)
            test_tot+=len(x_test)
        test_loss.append(loss_sum/test_tot)
    print('epochs={},  train_loss={},  test_loss={}'.format(ep,train_loss[-1],test_loss[-1]))
    if ep%200==0:
        lr*=0.8
        for p in opti.param_groups:
            p['lr']=lr
        torch.save(model,'./pkls/Tanhshrinkx'+str(ep)+'.pkl')
        np.savez('./npzs/Tanhshrinkx'+str(ep)+'.npz',train_loss=train_loss,test_loss=test_loss)
plt.plot(train_loss,label='train_loss')
plt.plot(test_loss,label='test_loss')
plt.yscale('log')

In [ ]:
x,y=1,1
for a,b in test:
    x,y=a,b
    break
fig,ax=plt.subplots(1,4,figsize=(12,3))
for i,axi in enumerate(ax.flat):
    axi.plot(WL*1e9,model(x).cpu().detach().numpy()[i]*100,label='DL')
    axi.plot(WL*1e9,y[i].cpu()*100,label='TMM')
    axi.legend()
fig.tight_layout()
plt.savefig('./figs/figTanhshrinktest.svg',format='svg')

In [ ]:
x,y=1,1
for a,b in train:
    x,y=a,b
    break
fig,ax=plt.subplots(1,4,figsize=(12,3))
for i,axi in enumerate(ax.flat):
    axi.plot(WL*1e9,model(x).cpu().detach().numpy()[i]*100,label='DL')
    axi.plot(WL*1e9,y[i].cpu()*100,label='TMM')
    axi.legend()
fig.tight_layout()
plt.savefig('./figs/figTanhshrinktrain.svg',format='svg')